### In this notebook, I first generate faces using captions from COCO dataset using ERNIE-ViLG, and then, I using face detector to extract faces from the images generated. 

In [1]:
#!pip install translators --upgrade

Load caption data

In [2]:
import paddlehub as hub
import translators as ts
lines = open('face_captions.txt').read().splitlines()

/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/image_utils.py:213: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  resample=Image.BILINEAR,
/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/image_utils.py:379: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  resample=Image.NEAREST,
/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/ernie_vil/feature_extraction.py:65: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  resample=Image.BICUBIC,
/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/clip/feature_extraction.py:64: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10

In [3]:
import random
random.seed(42)
#Generate 1000 random numbers between 0 and 9999
# I plan to generate about 600 images, but may need to come back generate more if some of the images do not contain a good face
randomlist = random.sample(range(0, 9999), 1000)

In [4]:
len(randomlist)

1000

In [5]:
randomlist[:10]

[1824, 409, 4506, 4012, 3657, 2286, 1679, 8935, 1424, 9674]

Start generating face images

In [9]:
import paddlehub as hub
import translators as ts
module = hub.Module(name="ernie_vilg")
for i in range(521, 600):
    prompt = ts.baidu(lines[randomlist[i]], professional_field='common')
    try: # for certain prompts, error message "暂不支持创作该内容，请修改后再试 (does not support this prompt, please modify)" was returned
        module.generate_image(text_prompts=prompt, style='写实风格',  output_dir='./temp_generated_face_image_holder/',topk=1)
        print(i)      
    except:
        continue

日访问量超限
日访问量超限


KeyboardInterrupt: 

In [10]:
i

521

extract faces from the generated face images 

In [9]:
#set up folders for extracted faces 
# initialize COCO API for instance annotations
import os
dataDir = './'

image_folder = "extracted_generated_faces"

savedirimage = dataDir + image_folder



if not os.path.exists(savedirimage):
	os.makedirs(savedirimage)

In [7]:
# install all packages
from keras.models import load_model
!pip install mtcnn
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

import numpy as np
from mtcnn.mtcnn import MTCNN
from numpy import asarray
from PIL import Image
import matplotlib.pyplot as plt


# Simple python package to shut up Tensorflow warnings and logs.
!pip install silence_tensorflow
import silence_tensorflow.auto

0.1.0


I did not use threshold as filtering criteria because this will gaurantee low FID score later. I still kept the dimension requirement. 

In [13]:
# extract a single face from a given photograph
def extract_face(filename, index, required_size=(250, 250)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    if len(results) >=1:
        x1, y1, width, height = results[0]['box']
        if height-width>=15:
            # bug fix
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            # extract the face
            face = pixels[y1:y2, x1:x2]
            # resize pixels to the model size
            image = Image.fromarray(face)
            image = image.resize(required_size)
            face_array = np.asarray(image)
            plt.imsave(os.path.join(savedirimage, 'face_img_{}.jpg'.format(index)),face_array)
            return True
    return False

In [14]:
files = os.listdir('temp_generated_face_image_holder/')

In [15]:
index =361
for file in files:
    if extract_face(os.path.join('./generated_face_images_with_ERNIE-ViLG/',file), index = index):
        print(index)
        index+=1
        
    

1/1 [==============================] - 0s 128ms/step
0
1/1 [==============================] - 0s 131ms/step
1
1/1 [==============================] - 0s 148ms/step
2
1/1 [==============================] - 0s 126ms/step
3
1/1 [==============================] - 0s 122ms/step
4
1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 122ms/step
5
1/1 [==============================] - 0s 123ms/step
6
1/1 [==============================] - 0s 127ms/step
7
1/1 [==============================] - 0s 128ms/step
8
1/1 [==============================] - 0s 126ms/step
9
1/1 [==============================] - 0s 152ms/step
10
1/1 [==============================] - 0s 141ms/step
11
1/1 [==============================] - 0s 124ms/step
12
1/1 [==============================] - 0s 123ms/step
13
1/1 [==============================] - 0s 122ms/step
14
1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 120ms/step
15
1/1 [==============================] - 0s 125ms/step
16
1/1 [==============================] - 0s 124ms/step
17
1/1 [==============================] - 0s 122ms/step
18
1/1 [==============================] - 0s 151ms/step
19
1/1 [==============================] - 0s 141ms/step
20
1/1 [==============================] - 0s 136ms/step
21
1/1 [==============================] - 0s 130ms/step
22
1/1 [==============================] - 0s 127ms/step
23
1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 124ms/step
24
1/1 [==============================] - 0s 120ms/step
25
1/1 [==============================] - 0s 127ms/step
26
1/1 [==============================] - 0s 162ms/step
27
1/1 [==============================] - 0s 143ms/step
28
1/1 [==============================] - 0s 130ms/step
29
1/1 [==============================] - 0s 123ms/step
30
1/1 [==============================] - 0s 120ms/step
31
1/1 [==============================] - 0s 129ms/step
32
3/3 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 165ms/step
33
1/1 [==============================] - 0s 145ms/step
34
1/1 [==============================] - 0s 139ms/step
35
1/1 [==============================] - 0s 137ms/step
36
1/1 [==============================] - 0s 126ms/step
37
1/1 [==============================] - 0s 125ms/step
38
1/1 [==============================] - 0s 136ms/step
39
1/1 [==============================] - 0s 123ms/step
40
1/1 [==============================] - 0s 123ms/step
41
1/1 [==============================] - 0s 145ms/step


...

1/1 [==============================] - 0s 164ms/step
328
1/1 [==============================] - 0s 146ms/step
329
1/1 [==============================] - 0s 133ms/step
330
1/1 [==============================] - 0s 128ms/step
331
1/1 [==============================] - 0s 121ms/step
332
1/1 [==============================] - 0s 124ms/step
333
1/1 [==============================] - 0s 134ms/step
334
2/2 [==============================] - 0s 14ms/step
335
1/1 [==============================] - 0s 165ms/step
336
1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 148ms/step
337
1/1 [==============================] - 0s 136ms/step
338
1/1 [==============================] - 0s 132ms/step
339
1/1 [==============================] - 0s 143ms/step
340
1/1 [==============================] - 0s 122ms/step
341
1/1 [==============================] - 0s 121ms/step
342
1/1 [==============================] - 0s 126ms/step
343
1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 147ms/step
344
1/1 [==============================] - 0s 127ms/step
345
1/1 [==============================] - 0s 123ms/step
346
1/1 [==============================] - 0s 130ms/step
347
1/1 [==============================] - 0s 122ms/step
348
1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 130ms/step
349
1/1 [==============================] - 0s 125ms/step
350
1/1 [==============================] - 0s 131ms/step
351
1/1 [==============================] - 0s 123ms/step
352
1/1 [==============================] - 0s 126ms/step
353
1/1 [==============================] - 0s 166ms/step
354
1/1 [==============================] - 0s 148ms/step
355
1/1 [==============================] - 0s 129ms/step
356
1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 128ms/step
357
1/1 [==============================] - 0s 125ms/step
358
1/1 [==============================] - 0s 124ms/step
359
1/1 [==============================] - 0s 132ms/step
360
